### Scripts for preprocessing all log-files
Author: Amer Altizini
Date: 06.12.2022
Information: Cells that are within the same section need to run subsequently

In [27]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import folium
import os
import glob

***
### Plotting Data
***

In [33]:
filepath_sensors = './Data/input/'
output_filepath = './Data/output/'

# speech_column = ['NTPTime','speech','manipulation','robot']
# gesture_column = ['NTPTime','gesture','manipulation','robot']
#
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [47]:
for participant_filepath in glob.glob(filepath_sensors + '*/*/', recursive=True):
    data_speech = pd.read_csv('{}Accelerometer.csv'.format(participant_filepath))
    display(data_speech)
    # print("{}.csv".format(filepath_sensors))
    # extract participants id
    components = os.path.dirname(participant_filepath)
    components = components.split(os.path.sep)
    participant = components[-1]
    display(participant)
    # participant_filepath = output_filepath + participant + '/'

,time,seconds_elapsed,z,y,x
0,1670350768736621800,0.301622,0.000000,0.000000,0.000000
1,1670350768746593300,0.311593,-0.037347,-0.002901,0.005993
2,1670350768756671500,0.321671,-0.082677,0.022784,0.017380
3,1670350768766749700,0.331750,-0.150242,0.028249,-0.006868
4,1670350768776924400,0.341924,-0.248892,-0.003977,-0.054344
...,...,...,...,...,...
1511,1670350783739272000,15.304272,-0.173714,0.044398,-0.158180
1512,1670350783749197000,15.314197,0.048381,0.001959,-0.115053
1513,1670350783759122400,15.324122,0.215286,-0.045304,0.045008
1514,1670350783769047800,15.334048,0.356706,-0.065271,0.107345


'2022-12-06_17-32-02'

In [25]:
# path_E4 = "./data/output/"
df = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')

df.index = pd.to_datetime(df['time'], unit = 'ns')

fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df.index, y = df[axis], name = axis))

fig.show()

In [10]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Gyroscope.csv')

df.index = pd.to_datetime(df['time'], unit = 'ns')

fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df.index, y = df[axis], name = axis))

fig.show()

In [11]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Magnetometer.csv')

df.index = pd.to_datetime(df['time'], unit = 'ns')

fig = go.Figure()

for axis in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x = df.index, y = df[axis], name = axis))

fig.show()

In [7]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Location.csv')
df.index = pd.to_datetime(df['time'], unit='ns')

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['speed'], mode='markers',
              error_y={'type': 'data', 'array': df['speedAccuracy']}))

fig.show()

In [23]:
df = pd.read_csv("./Data/2022-12-06_17-32-02/Location.csv")
coords = [(row.latitude, row.longitude) for _, row in df.iterrows()]

my_map = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=16)
folium.PolyLine(coords, color="blue", weight=5.0).add_to(my_map)
my_map

***
### Aligning & Interpolating Measurements Across Sensors
***

In [16]:
df_gyro = pd.read_csv('./Data/2022-12-06_17-32-02/Gyroscope.csv')
df_acce = pd.read_csv('./Data/2022-12-06_17-32-02/Accelerometer.csv')

for df in [df_gyro, df_acce]:
    df.index = pd.to_datetime(df['time'], unit = 'ns')

df = df_gyro.join(df_acce, lsuffix = '_gyro', rsuffix = '_acce', how = 'outer').interpolate()
df

,time_gyro,seconds_elapsed_gyro,z_gyro,y_gyro,x_gyro,time_acce,seconds_elapsed_acce,z_acce,y_acce,x_acce
time,,,,,,,,,,
2022-12-06 18:19:28.736621800,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.301622,0.000000,0.000000,0.000000
2022-12-06 18:19:28.746593300,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.311593,-0.037347,-0.002901,0.005993
2022-12-06 18:19:28.756671500,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.321671,-0.082677,0.022784,0.017380
2022-12-06 18:19:28.766749700,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.331750,-0.150242,0.028249,-0.006868
2022-12-06 18:19:28.776924400,NaN,NaN,NaN,NaN,NaN,1.670351e+18,0.341924,-0.248892,-0.003977,-0.054344
...,...,...,...,...,...,...,...,...,...,...
2022-12-06 18:19:43.798824000,1.670351e+18,15.363824,0.013715,0.048468,-0.000399,1.670351e+18,15.343973,0.327040,-0.105531,0.047844
2022-12-06 18:19:43.808749600,1.670351e+18,15.373750,0.021305,0.063381,-0.016378,1.670351e+18,15.343973,0.327040,-0.105531,0.047844
2022-12-06 18:19:43.818675000,1.670351e+18,15.383675,0.028761,0.081490,-0.043009,1.670351e+18,15.343973,0.327040,-0.105531,0.047844


***
### Fourier Transforms
***

In [18]:
df = pd.read_csv('./Data/2022-12-06_17-32-02/Gravity.csv')
ts = pd.to_datetime(df['time'], unit = 'ns')
period = np.nanmedian(ts.diff().dt.total_seconds())

sp = np.fft.fft(df['x'])
freq = np.fft.fftfreq(len(df.index), period)

mask = (freq >= 0) &(freq < 1)

fig = go.Figure()
fig.add_trace(go.Scatter(x = freq[mask], y = [np.linalg.norm(s) for s in sp[mask]]))
fig.show()